## 8 - Smart Pointers: `shared_ptr`
##### **Author: Adam Gatt**

The second type of smart pointer is `shared_ptr` which allows for multiple "owning" references, and the dynamic data will be deleted when the last owning reference falls out of scope. This is achieved by a reference-counting approach (much like garbage collection in the JVM). It is useful when:

* It cannot be made clear who the single authoritative owner should be
* There is a need for multiple references but no guarantees on which of them will be finalised last (e.g. a multi-threaded environment)

The behaviour of `shared_ptr` is achieved by a reference-counting approach, much like garbage collection in the JVM. This comes at a cost compared to its unique counterpart:

* The reference counter requires extra memory and prevents the smart pointer from being fully optimised away
* It is possible to create "reference cycles" of two shared_ptrs with (perhaps indirect) references to each other, preventing either of them from being cleaned up

As such, `shared_ptr` should not be used as a default approach where a `unique_ptr` would do instead. 

In [1]:
#include <memory>
#include <iostream>

class DynamicData {
    public:
    DynamicData() {
        std::cout << "Data created" << std::endl;
    }
    
    ~DynamicData() {
        std::cout << "Data cleaned up" << std::endl;
    }
}

In [2]:
{
    std::cout << "> outer scope" << std::endl;
    
    // First shared pointer, but contains no data yet
    std::shared_ptr<DynamicData> ptr1;
    
    {
        std::cout << "> inner scope" << std::endl;
        
        // Shared pointer, creating dynamic data
        std::shared_ptr<DynamicData> ptr2 = std::make_shared<DynamicData>();
    
        // Share ptr2 reference with ptr1 (via copy assignment operator)
        ptr1 = ptr2;
        std::cout << "< inner scope" << std::endl;
    }
    
    std::cout << "< outer scope" << std::endl;
}

> outer scope
> inner scope
Data created
< inner scope
< outer scope
Data cleaned up


#### Copy and move assignment 
As we can see, the copy constructor of a `shared_ptr` is not deleted like it is for `unique_ptr`. This allows us to write an assignment operation like
 * `mySharedPtr2 = mySharedPtr1`, or
 * `std::shared_ptr<Widget> newPtr{oldPtr}`
 
This operation creates a an additional reference to an existing `shared_ptr`, calling either the copy-assignment operator or copy constructor.

We do not have to call `std::move` when assigning our new `shared_ptr`. In fact, if we do then it is a different operation, a _"move assignment"_, instructing the new pointer to replace the old one. In this case, the previous `shared_ptr` has had its ownership of the data taken over by the new pointer, and the old pointer enters a "valid but unspecified state". 

In [ ]:
std::shared_ptr<Widget> newPointer = std::move(oldPointer);

// 1) newPointer has taken over oldPointer's ownership of the dynamic Widget object
// 2) The reference count is not increased
// 3) We must not use oldPointer again after this statement

#### Creation
`std::make_shared` is the preferred way to create a (rvalued) `shared_ptr`, just like `make_unique` for `unique_ptr`.
> `std::shared_ptr<Widget> myPointer = std::make_shared<Widget>(params);`

Some coders prefer to save on the redundant typing by making use of `auto` for the variable declaration. I can understand this and can see little risk in doing so, even if I personally prefer to type it out.
> `auto myPointer = std::make_shared<Widget>(params);`

An alternative syntax is to create the dynamic object ourselves and pass it to the constructor of the `shared_ptr`. Although it is an option, it is generally accepted as the less-preferred option for a reason that is explored at the end of this notebook.
> `std::shared_ptr<Widget> myPointer{new Widget()};`

#### Counting references
We can call `use_count()` to determine how many shared_ptrs are currently pointing to the same data. Note that only one constructor and one destructor has been called; the lines below don't show four instances of the dynamic data but four references to the same single instance.

In [4]:
{
    std::shared_ptr<DynamicData> ptr1 = std::make_shared<DynamicData>();
    std::shared_ptr<DynamicData> ptr2 = ptr1;
    std::shared_ptr<DynamicData> ptr3 = ptr1;
    std::shared_ptr<DynamicData> ptr4 = ptr1;

    std::cout << ptr1.use_count() << std::endl;
}

Data created
4
Data cleaned up


#### Dropping a reference
If ever we need to manually release ownership of the dynamic data we can call the `reset()` operation, which also offers overloads to [replace the ownership with a new dynamic object](https://en.cppreference.com/w/cpp/memory/shared_ptr/reset). This should probably only be used in extreme edge cases, as this would arguably run against the RAII philosophy of smart pointers employing a scope-based management of data ownership rather than having to write manual statements for releasing memory. 

In [3]:
{
    std::shared_ptr<DynamicData> myPointer = std::make_shared<DynamicData>();
    // I want to release ownership early
    myPointer.reset();
    std::cout << "Before the end of the scope block" << std::endl;
}

Data created
Data cleaned up
Before the end of the scope block


_Note that the `shared_ptr` will have its destructor called at the end of the scope block as usual, but as it has released its ownership and is now pointing at nothing, no operation will happen. It is always safe to have an "empty" smart pointer reach the end of its scope and get deleted._

### `weak_ptr` - a non-owning reference

Below is an example of a reference cycle that prevents two shared_ptrs from being deleted:

In [1]:
#include <memory>
#include <iostream>

class Manager;

class Employee {
    private:
    std::shared_ptr<Manager> manager;
    
    public:
    Employee(std::shared_ptr<Manager> manager) : manager(manager) {
        std::cout << "Employee created" << std::endl;
    }
    
    ~Employee() {
        std::cout << "Employee destroyed" << std::endl;
    }
}

In [2]:
class Manager {
    private:
    std::vector<std::shared_ptr<Employee>> employees;
    
    public:
    Manager(){
        std::cout << "Manager created" << std::endl;
    }
    
    ~Manager() {
        std::cout << "Manager destroyed" << std::endl;
    }
    
    void addEmployee(const std::shared_ptr<Employee>& newEmployee) {
        this->employees.push_back(newEmployee);
    }
}

In [3]:
{
    std::shared_ptr<Manager> manager = std::make_unique<Manager>();
    std::shared_ptr<Employee> employee = std::make_unique<Employee>(manager);
    manager->addEmployee(employee);
}

Manager created
Employee created


Both smart pointers have left the scope but the destructors are never called. To be clear, the _shared_ptr objects_ themselves are deleted, but the dynamic objects they point to persist. The shared_ptrs check their reference counts and see that it will not reach zero and so the dynamic data should not be deleted as other shared pointers may be depending on it. For each dynamic object, the two references are:
 - The shared_ptr object within the scope block, and
 - The shared_ptr in the other dynamic object that also refers to it
 
`std::weak_ptr` is a tool designed to hold a shared reference to dynamic memory without adding to the reference count. This makes it useful for breaking the reference cycle problem:

In [2]:
class Manager {
    private:
    // We now only want non-counted references to our employees
    std::vector<std::weak_ptr<Employee>> employees;
    
    public:
    Manager(){
        std::cout << "Manager created" << std::endl;
    }
    
    ~Manager() {
        std::cout << "Manager destroyed" << std::endl;
    }
    
    void addEmployee(const std::shared_ptr<Employee>& newEmployee) {
        // We receive a shared_ptr but want to add it to a vector of weak_ptrs. This is allowed
        // as there is an implicit assignment conversion between the two.
        this->employees.push_back(newEmployee);
    }
}

In [3]:
{
    std::shared_ptr<Manager> manager = std::make_unique<Manager>();
    std::shared_ptr<Employee> employee = std::make_unique<Employee>(manager);
    manager->addEmployee(employee);
}

Manager created
Employee created
Employee destroyed
Manager destroyed


`weak_ptr`s are also useful for references that are intended to be temporary by design, such as a cache, or an [Observer Pattern](https://en.wikipedia.org/wiki/Observer_pattern) list of subscribers.

In [ ]:
#include <unordered_map>
using Cache_t = std::unordered_map<Parameters, std::weak_ptr<Solution>>;
Cache_t cache;

std::shared_ptr<Solution> calculateSolution(const Parameters& params, Cache_t& cache) {
    
    // If we already calculated a solution ...
    if (cache.count(params) > 0) {
        
        std::weak_ptr<Solution>& existingSolution = cache[params];
        // ... and if the solution object still exists
        if (!existingSolution.expired()) {
            return existingSolution.lock();
        }
    }
    
    std::shared_ptr<Solution> newSolution = costlyCalculation(params);
    cache[params] = newSolution;
    return newSolution;
}

Because a `weak_ptr` does not add to the reference counter, there is the possibility that its underlying dynamic data has been deleted, if all of the owning shared_ptrs are gone. For full safety, `weak_ptr` has access to the [`expired()`](https://en.cppreference.com/w/cpp/memory/weak_ptr/expired) method to determine if the dynamic data still exists and is safe to use. Alternatively, it has access to the same `use_count()` method, being careful to specify that this still reports on the number of `shared_ptr`s referencing the object and not the number of `weak_ptr`s.

But note that [`std::weak_ptr`](https://en.cppreference.com/w/cpp/memory/weak_ptr) doesn't have an implementation of `get()`, `operator*` or `operator->` (or `operator[]`). In order to access the dynamic data we need to call [`lock()`](https://en.cppreference.com/w/cpp/memory/weak_ptr/lock) to create a new `shared_ptr` instance to make use of.

### Rules for using shared / weak pointers
* Create shared_ptrs with the `std::make_shared` function
* Assign to another shared_ptr or pass to function to add to shared ownership
* Use `std::move` when assigning or passing to function to transfer ownership without increasing reference count
* Similar same rules for function parameters as `unique_ptr`:
  * `std::shared_ptr<Data>` if we want the function to add to the ownership of the data
  * `std::shared_ptr<Data>&` if we want to make changes with the ownership of the provided shared_ptr
  * `Data&` / `Data*` / `Data` if we want access to the underlying data (or a copy) without adding to ownership


* Use `std::weak_ptr` for temporary, non-owning reference, or to break reference cycles
* Promote weak_ptr to shared_ptr with `lock()` in order to safely use dynamic data
* Check if the dynamic object still exists with `expired()`, or by inspecting the contents of the `std::shared_ptr` created when calling `lock()`

### Optional: Smart pointer creation, `new` vs `make_*`
In the examples above I often used the `std::make_unique` function to create unique_ptrs. If I wanted to, there is no reason why I can't create a unique_ptr with [its standard constructor](https://en.cppreference.com/w/cpp/memory/unique_ptr/unique_ptr), passing it a pointer to the dynamic data that I create manually with the usual `new` keyword.

In [ ]:
#include <memory>
#include <string>

std::unique_ptr<std::string> dynamicStringManager(new std::string("Hello, World!"));

*dynamicStringManager

However, in C++14 the functions `make_unique` and `make_shared` were introduced as it was realised that there was a subtle bug that may result in a memory leak. In short, if a smart pointer was created as a temporary object (e.g. such as in a function call) and then an exception occurs before the temporary object is used and resolved, there is a chance that the dynamic data would be successfully created but the creation of the owning smart pointer would fail. With no smart pointer to clean it up, the dynamic data would leak.

In [ ]:
func1(std::unique_ptr<A>(new A("Hello", 50)), func2())

Consider the (contrived) example above, with a function "func1" that requires a smart pointer and the results of a second function "func2". In the C++ standard, the execution order of the statement is not strongly defined. It is very possible for the dynamic object `new A()` to be created first, and then for `func2` to execute and cause an exception. The temporary unique_ptr never gets created, and there is nothing responsible for deleting our dynamic object.

In the alternative syntax below, the `make_unique()` call is atomic. The unique_ptr may already have been created when func2 throws an exception, but it is firmly an existing _temporary object_ by then. When the exception occurs the stack will be unwound and the smart pointer's destructor will be called, cleaning up the data. 

In [ ]:
func1(std::make_unique<A>("Hello", 50), func2())

But then when C++17 was released, the standard was changed to include additional guarantees, including that all side-effects of a function argument must be evaluated before moving on to the next argument. This rule fixes the above example and obviates the safety-based argument for using make_unique. However, if you cannot guarantee that your code will only be compiled with C++17, it may be good to make a habit of using make_unique (and make_shared) to avoid being caught out.

In addition to the safety argument, there are two other arguments commonly made for using the `make_*()` creation functions:
* You only need to write the underlying type name once, improving readability when the type name is especially long
* You can arguably avoid the need to write `new` completely, making the new/delete semantics effectively obsolete.